In [ ]:
#uncomment if pdal is not already install 
#pip install pdal


In [1]:
import os
import glob
import pdal


In [2]:
def rasterize_ground_points(input_directory, output_path, resolution):
    # Get a list of all csv files in the input directory
    csv_files = glob.glob(os.path.join(input_directory, "*.csv"))

    # Create the PDAL pipeline
    pipeline = pdal.Pipeline()

    # Set the input files as a merged view
    pipeline.set_input(pdal.merge(*csv_files))

    # Set the filters for ground segmentation and rasterization
    pipeline.set_filters([

        # Filters out points with 0 returns
        {"type": "filters.mongo",
        "expression": {"$and": [{"ReturnNumber": {"$gt": 0}},{"NumberOfReturns": {"$gt": 0}}]}
        },

        # Extended Local Minimum filter
        {"type": "filters.elm"
        },

        # Outlier filter
        {"type": "filters.outlier",
        "method": "statistical",
        "mean_k": 12,
        "multiplier": 2.2},

        # SMRF classifier for ground
        {
            "type": "filters.smrf",
            "ignore": "Classification[7:7], NumberOfReturns[0:0], ReturnNumber[0:0]"
        },
        # Select ground points only
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"
        },
        # Write TIF file
        {
            "type": "writers.gdal",
            "filename": output_path,
            "resolution": resolution,
            "output_type": "idw"
        }
    ])

    # Execute the pipeline
    pipeline.execute()


In [3]:
input_directory = "Sites/RCEW/IS2_Data/" #foler containing the laz files
output_path = "Sites/RCEW/IS2_Data/RCEW-ICESat2-ATL03.tif" #resulting tif file
resolution = 1.0  # Adjust the resolution as needed

rasterize_ground_points(input_directory, output_path, resolution)


AttributeError: 'Pipeline' object has no attribute 'set_input'

In [ ]:
# Get a list of all csv files in the input directory
csv_files = glob.glob(os.path.join(input_directory, "*.csv"))

# Create the PDAL pipeline
pipeline = pdal.Pipeline()

    # Set the input files as a merged view
pipeline.set_input(pdal.merge(*csv_files))

In [ ]:
data = "Sites/RCEW/IS2_Data/RCEW-ICESat2-ATL03.las"

pipeline = pdal.Reader.las(filename=data).pipeline()
print(pipeline.execute())